# 自动站数据的类


In [2]:
%%time
import pymssql 
import pandas as pd

## sqlserver
server = "172.21.158.201"    # 连接服务器地址
user = "down"# 连接帐号
password = "downx"# 连接密码
conn = pymssql.connect(server, user, password, "ZJSZDZDB")  #获取连接

sql_location = "select b.StationName,b.county,b.Town,b.lat,b.Height,b.lon,b.IIiii,tTime,dFy,fFy,T,Tx,Tn,VV,RR from TAB_Mws2022 as a left join TAB_StationInfo as b on a.IIiii=b.IIiii where\
(b.IIiii in (select IIiii from TAB_StationInfo where(City = '台州') and tTime between '2022-01-25 20:00' and '2022-02-10 20:00'))"


df_location = pd.read_sql(sql_location , con=conn)
df_location

CPU times: user 1.49 s, sys: 90 ms, total: 1.58 s
Wall time: 5.57 s


,StationName,county,Town,lat,Height,lon,IIiii,tTime,dFy,fFy,T,Tx,Tn,VV,RR
0,街头雪深,天台县,街头镇,29.12444,01140,120.7933,K8734,2022-01-25 23:00:00,-9999,-9999,-9999,-9999,-9999,-9999,0
1,后岸,天台县,街头镇,29.07861,01010,120.8139,K8748,2022-01-25 23:00:00,16,42,73,77,73,-9999,4
2,欢岙,天台县,坦头镇,29.18444,02910,121.1128,K8773,2022-01-25 23:00:00,180,18,59,60,59,-9999,5
3,济溪,天台县,街头镇,29.05389,01990,120.8553,K8790,2022-01-25 23:00:00,-9999,-9999,72,73,71,-9999,6
4,叶家村,三门县,海润街道,29.06750,00520,121.4881,K8812,2022-01-25 23:00:00,315,19,75,79,75,-9999,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176568,溪港,仙居县,溪港乡,28.57361,02200,120.3961,K8912,2022-02-10 19:00:00,0,2,81,86,81,-9999,0
176569,四小,仙居县,安洲街道,28.87278,00760,120.7306,K8927,2022-02-10 19:00:00,95,16,80,85,80,-9999,0
176570,山岙雪深,仙居县,横溪镇,28.77944,05540,120.4283,K8932,2022-02-10 19:00:00,-9999,-9999,-9999,-9999,-9999,-9999,0
176571,船山村,仙居县,南峰街道,28.81833,01860,120.7011,K8973,2022-02-10 19:00:00,68,43,72,75,72,-9999,0


In [3]:
df_location.to_csv('data_zdz_height.csv')

In [10]:
%%time
import gevent
from math import isnan
import numpy as np
#import modin.pandas as pd 
import pandas as pd
import time
from func import *
import matplotlib
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import json
import math

import pymssql 
import datetime
class zdz_data:
    def __init__(self, start,end):
        self.start = start
        self.end = end
        # 基础数据
        self.station_all = self.read_csv()
        self.event_data()
        self.rain_data(start,end)
    def sql_date(self):
        '''数据库读取sql数据'''
        print('读取数据库数据')
    def read_csv(self):
        '''
        1.数据读取---此方法为测试方法
        2.数据分段---每天
        3.每天统计
        '''
#         station_Mws = pd.read_csv("Mws_15.csv")
#         station_Aws = pd.read_csv("Aws_15.csv")
#         station_all = pd.concat([station_Aws, station_Mws])
        station_all = pd.read_csv("data_zdz_height.csv")
        return station_all
    def event_data(self):
        '''
        1.数据库查询
        2.数据分段---每天
        3.每天统计
        ['2022-04-14', 200,'高温 浓雾', '降水 大风', 200],
        '''
        dateList = [ ] 
        # 多少天
        dates = self.return_daylist()
        for i in dates:
            #解析每天的数据
            # grouped_county = self.station_all.groupby('county')
            # grouped_IIiii = self.station_all.groupby('IIiii')
            # for i in self.grouped_county.size().index: 
            s_date = i + ' ' + '00:00'
            e_date = i + ' ' + '23:00'
            df_preday = self.station_all[(self.station_all['tTime'] >= s_date) & (self.station_all['tTime'] <= e_date)]              
#             df_RR= df_preday[df_preday['RR'] == -9999 ].count()
#             grouped_county = 
            date_preday = []
            # 日期
            day_date = str(i)
            # 面雨量
            rainfall = 200
            # 气温
            temp_min = df_preday[(df_preday['Height']<6000)&(df_preday['T'] > -400)]['T'].min()
            temp_min = temp_min/10.0
            temp_max = df_preday[(df_preday['Height']<6000)&(df_preday['T'] > -400)]['T'].max()
            temp_max = temp_max/10.0
            temp_label = ''
            if temp_min < 3.0:
                temp_label = '低温'
            elif temp_max > 35.0:
                temp_label = '高温'
            else:
                temp_label = ' '
            # 能见度
            # df_VV= df_preday[(df_preday['VV']>0.1)&(df_preday['VV']<500)]['VV'].count()
            vv = df_preday[(df_preday['VV']>0.1)&(df_preday['VV']<500)]['VV'].count()
            if vv>0:
                view_label = '浓雾'
            else:
                view_label = ' '    
            # 降水
            df_RR= df_preday[(df_preday['RR']>0.3)&(df_preday['RR']<8888)]['RR'].count()
            
            if df_RR > 0:
                pre_label = '降水'
            else:
                pre_label = ' '
            # 大风
            wind_count = df_preday[df_preday['fFy']>187]['fFy'].count()
            if wind_count>0:
                wind_label = '大风'
            else:
                wind_label = ' '
            date_preday = [day_date,rainfall,temp_label + ' '+ view_label, pre_label +' '+ wind_label ,rainfall]
            dateList.append(date_preday)
#         print('event_data:返回一个每天的数组和日历所需的数据','气温',dateList)
    def return_daylist(self):
        '''
        返回每天的起始列表
        '''
        dates = []
#         dt = datetime.datetime.strptime(self.start, "%Y-%m-%d")
        dt = datetime.datetime.strptime(self.start[0:10], "%Y-%m-%d")
        date = self.start[:10]
        while date <= self.end[:10]:
            dates.append(date)
            dt = dt + datetime.timedelta(1)
            date = dt.strftime("%Y-%m-%d")
        return dates
    def rain_data(self,start,end):
        '''
        1.根据起始时间计算面雨量的sql语句
        2.根据初始语句返回每站的总降水量
        '''
        station_all = self.station_all
        
        data_rain = {
            'rain_sum':{
                'time':[],
                'data':[]
            },
            'IIiii_data':{}
            
        }
        grouped_tTime = station_all.groupby('tTime')
        for i in grouped_tTime.size().index:  
            data= grouped_tTime.get_group(i)
            data['RR'].replace(-9999,np.nan,inplace=True)
            rain_mean = data['RR'].mean()/10.0
            rain_time = i
            data_rain['rain_sum']['time'].append(rain_time)
            data_rain['rain_sum']['data'].append(rain_mean)
        # 导出单站数据
        grouped_IIiii = station_all.groupby('IIiii')
        for i in grouped_IIiii.size().index:  
            data= grouped_IIiii.get_group(i)
            data['RR'].replace(-9999,np.nan,inplace=True)
            station_name = str(i) 
            data_rain['IIiii_data'][station_name] = data
            
#         print('降水页面的数据处理',data_rain['IIiii_data'])
    def wind_data(self,start,end):
        '''
        1.根据sql语句计算8及以上大风的分布和排序
        '''
        print('风场的计算')
    def view_data(self,start,end):
        '''
        1.根据sql语句计算低能见度的分布和排序
        '''
        print('能见度的计算')
    def temp_data(self,start,end):
        '''
        1.根据sql语句计算高低温的分布和排序
        '''
        print('温度的计算')
    def pre_day(self):
        '''
        日报的响应时间、触发后统计对应天的灾情
        '''
        print('计算指定日期的数据')


    

start = '2022-01-25 20:00'
end = '2022-02-10 06:00'


# start = '2022-01-25'
# end = '2022-02-10'

sql_worker = zdz_data(start,end)
# a = sql_worker.read_csv()
# b = sql_worker.return_daylist()

CPU times: user 727 ms, sys: 6.49 ms, total: 734 ms
Wall time: 768 ms


In [17]:
b

['2022-01-25',
 '2022-01-26',
 '2022-01-27',
 '2022-01-28',
 '2022-01-29',
 '2022-01-30',
 '2022-01-31',
 '2022-02-01',
 '2022-02-02',
 '2022-02-03',
 '2022-02-04',
 '2022-02-05',
 '2022-02-06',
 '2022-02-07',
 '2022-02-08',
 '2022-02-09',
 '2022-02-10']

In [3]:
a

,Unnamed: 0,StationName,county,Town,lat,lon,IIiii,tTime,dFy,fFy,T,Tx,Tn,VV,RR
0,0,洪家,椒江区,洪家街道,28.61806,121.41639,58665,2022-01-25 20:00:00,50,32,101,102,101,5149,0
1,1,大陈,椒江区,大陈镇,28.44600,121.90333,58666,2022-01-25 20:00:00,26,134,92,94,92,4605,0
2,2,坎门,玉环市,坎门街道,28.07750,121.27083,58667,2022-01-25 20:00:00,20,78,93,93,92,5721,0
3,3,洪家,椒江区,洪家街道,28.61806,121.41639,58665,2022-01-25 21:00:00,61,27,99,101,99,4309,0
4,4,大陈,椒江区,大陈镇,28.44600,121.90333,58666,2022-01-25 21:00:00,31,113,92,93,92,5122,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170994,170994,柯思,仙居县,田市镇,28.66389,120.67080,K8908,2022-02-10 06:00:00,0,0,54,55,53,-9999,0
170995,170995,淡竹,仙居县,淡竹乡,28.67194,120.57060,K8911,2022-02-10 06:00:00,265,3,53,54,53,-9999,0
170996,170996,大陈村雪深,仙居县,埠头镇,28.85028,120.51170,K8931,2022-02-10 06:00:00,-9999,-9999,-9999,-9999,-9999,-9999,0
170997,170997,西陆,仙居县,下各镇,28.82278,120.84830,K8950,2022-02-10 06:00:00,130,8,56,57,56,-9999,0
